In [326]:
import pandas as pd
import sqlite3
import numpy as np


# get spotify credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotifyCred = pd.read_csv('spotifyCred.csv')

client_credentials_manager = SpotifyClientCredentials(client_id= spotifyCred['0'][0],
                                                     client_secret= spotifyCred['0'][1])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = sqlite3.connect('/Users/aaronlevi/Documents/sql_db/chords_list.db')
cur = conn.cursor()

In [338]:
# check if known-song & playlist is in db
# track = 'weezer, island in the sun'
track = 'jimi hendrix, all along the watchtower'

known_song   = str(track).split(', ')[1]
artist = str(track).split(', ')[0]
    
# get db...
song_info = pd.read_sql_query("SELECT * FROM basic_info", conn)
dblist    = song_info['Song'].tolist()
# song_info.drop('index', axis=1, inplace=True)
allFeatures = pd.read_sql_query("SELECT * FROM features", conn)    
# allFeatures.drop('index', axis=1, inplace=True)
isAdv = pd.read_sql_query("SELECT model_isAdv FROM label", conn) 

if known_song.replace(' ', '-').lower() in dblist:
    knownSong_ix = dblist.index(known_song.replace(' ', '-').lower())

    # get chords and features for the known song
    known_df = pd.DataFrame(song_info.loc[knownSong_ix]).transpose()
    known_df['isAdv'] = isAdv.loc[knownSong_ix][0]
    known_df['nChords'] = allFeatures['n_unique_chords'].loc[knownSong_ix]

known_df.drop('index', axis=1, inplace=True)
known_df.reset_index(drop=True, inplace=True)
print(known_df)

         Artist    Chords Label                      Song  isAdv  nChords
0  jimi-hendrix  Bb,Cm,Ab  None  all-along-the-watchtower      0        2


In [339]:
isAdv.to_csv('isAdv.csv', index=False)
allFeatures.to_csv('allFeatures.csv', index=False)
song_info.to_csv('song_info.csv', index=False)

In [340]:
# get playlist tracks
pl_link = 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=Lls1MGwsQRKr4BN64HBRZw'
    
pl_id   = pl_link.split('playlist/')[1]
pl_id   = pl_id.split('?')[0]

pl_tracks  = []
pl_artists = []
pl_indb    = []

results = sp.user_playlist('Spotify', pl_id,
                            fields='tracks,next,name')

# check if their in the db
for nItem in range(0, len(results['tracks']['items'])):
    pl_tracks.append(results['tracks']['items'][nItem]['track']['name'].split(' -')[0].replace(' ', '-').lower() )
    #     pl_tracks.append(results['tracks']['items'][nItem]['track']['name'].replace(' ', '-').lower())
    pl_artists.append(results['tracks']['items'][nItem]['track']['artists'][0]['name'].replace(' ', '-').lower())

    if pl_tracks[nItem] in dblist:
        pl_indb.append(dblist.index(pl_tracks[nItem]))

#     return pl_tracks, pl_artists
# SELECT EXISTS(SELECT 1 FROM myTbl WHERE u_tag="tag");

# pl_db_ix = [i for i, x in enumerate(pl_indb) if x]

In [341]:
# get chords and features for playlist songs in db
input_df = song_info.loc[pl_indb]
input_df['isAdv'] = isAdv.loc[pl_indb]
input_df['nChords'] = allFeatures['n_unique_chords'].loc[pl_indb]

input_df.drop('index', axis=1, inplace=True)
input_df.reset_index(drop=True, inplace=True)

# input_df

In [342]:
# classify any songs not in db? 
# pd.read_sql_query("SELECT * FROM basic_info WHERE Song='island-in-the-sun'", conn)
# pd.read_sql_query("SELECT EXISTS(SELECT 1 FROM basic_info WHERE Song= str(pl_tracks[0]) ", conn)


In [343]:
# run chord similarity analysis of songs in same class
# known_df and input_df

# get index of playlist songs in same class
isAdvList = input_df['isAdv'].tolist()
targDifficulty = known_df['isAdv'][0]

if targDifficulty is 1:
    sameDiff = [i for i, x in enumerate(isAdvList) if x]
else:
    sameDiff = [i for i, x in enumerate(isAdvList) if not x]
    
# narrow to only sameDiff songs
sameDiff_df = input_df.loc[sameDiff]

# get a list of songs with the same or few n unique chords
close_n    = sameDiff_df['nChords'] <= known_df['nChords'][0]
close_n_ix = [i for i, x in enumerate(close_n) if x]

close_df = sameDiff_df.loc[close_n_ix]
close_df.reset_index(drop=True, inplace=True)
# close_df

# # get a list of songs with higher n unique chords
# far_n    = sameDiff_df['nChords'] > known_df['nChords'][0]
# far_n_ix = [i for i, x in enumerate(far_n) if x]

# far_df = sameDiff_df.loc[far_n_ix]
# far_df.reset_index(drop=True, inplace=True)

In [345]:
# far_df.dropna(axis=0, how='any', inplace=True)
# far_df

if known_df['Song'][0] in close_df['Song'].tolist():
    k_ix = close_df['Song'].tolist().index(known_df['Song'][0].replace(' ', '-').lower())

# print(k_ix)
# close_df
close_df.drop(k_ix, axis=0, inplace=True)
close_df.reset_index(drop=True, inplace=True)

In [310]:

known_chords_list = known_df['Chords'][0].split(',')

close_nSameChords = []
close_propSameChords = []

far_nSameChords = []
far_propSameChords = []

for iSong in range(0, len(close_df)):
#     if isinstance(close_df['Chords'][iSong], str):
    close_nSameChords.append( len([ele for ele in known_chords_list if(ele in close_df['Chords'][iSong].split(','))]) )
    close_propSameChords.append( len([ele for ele in known_chords_list if(ele in close_df['Chords'][iSong].split(','))]) / len(close_df['Chords'][iSong].split(','))  )
#     else:
#         nSameChords.append(0)
#         propSameChords.append(0)

# for iSong in range(0, len(far_df)):
# #     if isinstance(close_df['Chords'][iSong], str):
#     far_nSameChords.append( len([ele for ele in known_chords_list if(ele in far_df['Chords'][iSong].split(','))]) )
#     far_propSameChords.append( len([ele for ele in known_chords_list if(ele in far_df['Chords'][iSong].split(','))]) / len(far_df['Chords'][iSong].split(','))  )
# #     else:
# #         nSameChords.append(0)
# #         propSameChords.append(0)
        
maxOverlap = np.argmax(close_propSameChords)
minOverlap = np.argmin(close_propSameChords)

# far_maxOverlap = np.argmax(far_propSameChords)
# far_minOverlap = np.argmin(far_propSameChords)


easy_rec   = close_df.iloc[maxOverlap]
medium_rec = close_df.iloc[minOverlap]
# hard_rec   = far_df.iloc[far_maxOverlap]
# wayhard_rec   = far_df.iloc[far_minOverlap]

print(easy_rec)
print(medium_rec)
# print(hard_rec)
# print(wayhard_rec)



Artist          stevie-nicks
Chords                Em,D,C
Label                   None
Song       edge-of-seventeen
isAdv                      0
nChords                    2
Name: 16, dtype: object
Artist                 jimi-hendrix
Chords                     Bb,Cm,Ab
Label                          None
Song       all-along-the-watchtower
isAdv                             0
nChords                           2
Name: 1, dtype: object


In [309]:
easy_rec=pd.DataFrame(easy_rec).transpose()
easy_rec

,Artist,Chords,Label,Song,isAdv,nChords
16,stevie-nicks,"Em,D,C",None,edge-of-seventeen,0,2


In [ ]:
# # DO SAME FOR AN ADVANCED SONG
# # get index of playlist songs in same class
# isAdvList = input_df['isAdv'].tolist()
# targDifficulty = 1

# if targDifficulty is 1:
#     sameDiff = [i for i, x in enumerate(isAdvList) if x]
# else:
#     sameDiff = [i for i, x in enumerate(isAdvList) if not x]

# # narrow to only sameDiff songs
# sameDiff_df = input_df.loc[sameDiff]

# # get a list of songs with the same or few n unique chords
# close_n    = sameDiff_df['nChords'] <= known_df['nChords'][0]
# close_n_ix = [i for i, x in enumerate(close_n) if x]

# close_df = sameDiff_df.loc[close_n_ix]
# close_df.reset_index(drop=True, inplace=True)

# # close_df = input_df.reindex[close_n_ix]

# close_df.reset_index(drop=True, inplace=True)

# known_chords_list = known_df['Chords'][0].split(',')

# nSameChords = []
# propSameChords = []
# cnt = 1

# for iSong in range(0, len(close_df)):
#     if isinstance(close_df['Chords'][iSong], str):
#         nSameChords.append( len([ele for ele in known_chords_list if(ele in close_df['Chords'][iSong].split(','))]) )
#         propSameChords.append( len([ele for ele in known_chords_list if(ele in close_df['Chords'][iSong].split(','))]) / len(close_df['Chords'][iSong].split(','))  )
#     else:
#         nSameChords.append(0)
#         propSameChords.append(0)
        
# maxOverlap = np.argmax(propSameChords)

# hard_rec = close_df.iloc[12]
# print(hard_rec)

In [283]:
known_df['Chords'][0]

'Am,Em,D,C,G'

In [308]:
easy_rec['Song'].replace('-', ' ').title()

KeyError: 'Song'

In [312]:
easy_rec['Artist'].replace('-', ' ').title()

'Stevie Nicks'

In [311]:
medium_rec['Song'].replace('-', ' ').title()

'All Along The Watchtower'

In [288]:
medium_rec['Artist'].replace('-', ' ').title()

'Jimi Hendrix'

In [289]:
medium_rec['Chords']

'Bb,Cm,Ab'